# Data Augmentation Strategy - Expansión Inteligente del Dataset
**Sistema de Anotación de Video - Entrega 2**

Este notebook implementa estrategias de augmentation para balancear clases y expandir el dataset.

## Análisis del EDA:
- **Desbalance identificado:** Sentarse (14.1%) vs Girar (23.8%)
- **Objetivo:** +2,000 frames adicionales
- **Estrategia:** Multi-nivel (SMOTE + Espacial + Temporal + Ruido)

## Resultados esperados:
- Dataset balanceado (~6,500 frames)
- Mejora en robustez del modelo
- Validación de calidad sintética

---


## Paso 1: Instalación y Setup
Instalar librerías necesarias para data augmentation.


In [2]:
# Instalar dependencias para augmentation
!pip install pandas numpy matplotlib seaborn scikit-learn imbalanced-learn scipy

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imblearn # Import imblearn directly
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from scipy.interpolate import CubicSpline
from pathlib import Path
import json
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías para augmentation cargadas")
print(f"📊 Pandas: {pd.__version__}")
print(f"🔢 NumPy: {np.__version__}")
print(f"🔬 Imbalanced-learn: {imblearn.__version__}")

✅ Librerías para augmentation cargadas
📊 Pandas: 2.2.2
🔢 NumPy: 2.0.2
🔬 Imbalanced-learn: 0.14.0


## Paso 2: Cargar Dataset Base
Cargar el dataset limpio de la Entrega 1 para análisis de desbalance.


In [5]:
# CARGAR DATASET BASE DE ENTREGA 1
def load_base_dataset():
    """Cargar dataset base desde Entrega 1"""
    print("📂 CARGANDO DATASET BASE DESDE ENTREGA 1")
    print("=" * 50)

    # Ruta relativa desde Entrega2 hacia Entrega1 - Adjusting to load from current directory
    landmarks_path = Path(".") # Search in the current directory

    if not landmarks_path.exists():
        print(f"❌ No se encontró: {landmarks_path}")
        print("💡 Asegúrate de ejecutar desde Entrega2/notebooks/")
        return None

    # Cargar todos los CSVs ending with _landmarks.csv
    csv_files = list(landmarks_path.glob("*_landmarks.csv"))
    print(f"📁 Archivos encontrados: {len(csv_files)}")

    if not csv_files:
        print("❌ No se encontraron archivos CSV de landmarks en el directorio actual.")
        return None

    dataframes = []
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        dataframes.append(df)

    combined_df = pd.concat(dataframes, ignore_index=True)

    # Limpiar datos (remover frames sin detección)
    landmark_cols = [col for col in combined_df.columns
                    if col not in ['activity', 'video_file', 'frame_number']]

    # Solo frames con detección
    df_clean = combined_df[(combined_df[landmark_cols] != 0.0).any(axis=1)].copy()

    print(f"📊 DATASET CARGADO:")
    print(f"   Total frames: {len(df_clean):,}")
    print(f"   Videos: {df_clean['video_file'].nunique()}")
    print(f"   Actividades: {df_clean['activity'].nunique()}")

    # Mostrar distribución actual
    print(f"\n📈 DISTRIBUCIÓN ACTUAL:")
    activity_counts = df_clean['activity'].value_counts()
    for activity, count in activity_counts.items():
        pct = count / len(df_clean) * 100
        print(f"   {activity.replace('_', ' ').title()}: {count:,} frames ({pct:.1f}%)")

    balance_ratio = activity_counts.min() / activity_counts.max()
    print(f"\n⚖️ Balance actual: {balance_ratio:.2f}")

    return df_clean

# Cargar dataset base
base_df = load_base_dataset()

📂 CARGANDO DATASET BASE DESDE ENTREGA 1
📁 Archivos encontrados: 45
📊 DATASET CARGADO:
   Total frames: 4,575
   Videos: 45
   Actividades: 5

📈 DISTRIBUCIÓN ACTUAL:
   Girar: 1,089 frames (23.8%)
   Caminar Regreso: 1,041 frames (22.8%)
   Caminar Hacia: 991 frames (21.7%)
   Ponerse Pie: 809 frames (17.7%)
   Sentarse: 645 frames (14.1%)

⚖️ Balance actual: 0.59


## Paso 3: Análisis de Desbalance
Analizar en detalle el desbalance de clases y calcular targets de augmentation.


In [6]:
# ANÁLISIS DETALLADO DE DESBALANCE
def analyze_class_imbalance(df):
    """Analizar desbalance y calcular targets para augmentation"""
    print("⚖️ ANÁLISIS DETALLADO DE DESBALANCE DE CLASES")
    print("=" * 60)

    activity_counts = df['activity'].value_counts()
    target_frames = activity_counts.max()  # Usar la clase mayoritaria como target

    print(f"🎯 ANÁLISIS ACTUAL:")
    print(f"   Clase mayoritaria: {activity_counts.index[0]} ({activity_counts.iloc[0]:,} frames)")
    print(f"   Clase minoritaria: {activity_counts.index[-1]} ({activity_counts.iloc[-1]:,} frames)")
    print(f"   Ratio balance: {activity_counts.min()/activity_counts.max():.2f}")

    # Calcular cuántos frames necesita cada actividad
    augmentation_needed = {}
    total_augmentation = 0

    print(f"\n📊 PLAN DE BALANCEO (Target: {target_frames:,} frames por clase):")

    for activity, current_count in activity_counts.items():
        needed = max(0, target_frames - current_count)
        augmentation_needed[activity] = needed
        total_augmentation += needed

        status = "✅ BALANCEADA" if needed == 0 else f"📈 NECESITA +{needed:,}"
        print(f"   {activity.replace('_', ' ').title()}: {current_count:,} → {target_frames:,} ({status})")

    print(f"\n🎯 RESUMEN DE AUGMENTATION:")
    print(f"   Total frames a generar: {total_augmentation:,}")
    print(f"   Dataset final esperado: {len(df) + total_augmentation:,} frames")
    print(f"   Incremento: {(total_augmentation/len(df))*100:.1f}%")

    # Estrategia por actividad
    print(f"\n📋 ESTRATEGIA POR ACTIVIDAD:")
    for activity, needed in augmentation_needed.items():
        if needed > 0:
            # Distribuir técnicas
            smote_frames = min(needed, needed // 2)
            spatial_frames = needed // 4
            temporal_frames = needed // 4
            noise_frames = needed - smote_frames - spatial_frames - temporal_frames

            print(f"\n   🎯 {activity.replace('_', ' ').title()} (+{needed:,} frames):")
            print(f"      🔄 SMOTE: {smote_frames:,} frames")
            print(f"      🔀 Rotación espacial: {spatial_frames:,} frames")
            print(f"      ⏱️ Interpolación temporal: {temporal_frames:,} frames")
            print(f"      🎲 Noise injection: {noise_frames:,} frames")

    return augmentation_needed, target_frames

# Ejecutar análisis de desbalance
if base_df is not None:
    aug_needed, target_count = analyze_class_imbalance(base_df)
    print(f"\n✅ ESTRATEGIA DE AUGMENTATION DEFINIDA")
else:
    print("❌ No hay datos para analizar desbalance")


⚖️ ANÁLISIS DETALLADO DE DESBALANCE DE CLASES
🎯 ANÁLISIS ACTUAL:
   Clase mayoritaria: girar (1,089 frames)
   Clase minoritaria: sentarse (645 frames)
   Ratio balance: 0.59

📊 PLAN DE BALANCEO (Target: 1,089 frames por clase):
   Girar: 1,089 → 1,089 (✅ BALANCEADA)
   Caminar Regreso: 1,041 → 1,089 (📈 NECESITA +48)
   Caminar Hacia: 991 → 1,089 (📈 NECESITA +98)
   Ponerse Pie: 809 → 1,089 (📈 NECESITA +280)
   Sentarse: 645 → 1,089 (📈 NECESITA +444)

🎯 RESUMEN DE AUGMENTATION:
   Total frames a generar: 870
   Dataset final esperado: 5,445 frames
   Incremento: 19.0%

📋 ESTRATEGIA POR ACTIVIDAD:

   🎯 Caminar Regreso (+48 frames):
      🔄 SMOTE: 24 frames
      🔀 Rotación espacial: 12 frames
      ⏱️ Interpolación temporal: 12 frames
      🎲 Noise injection: 0 frames

   🎯 Caminar Hacia (+98 frames):
      🔄 SMOTE: 49 frames
      🔀 Rotación espacial: 24 frames
      ⏱️ Interpolación temporal: 24 frames
      🎲 Noise injection: 1 frames

   🎯 Ponerse Pie (+280 frames):
      🔄 SMOTE: 

## Paso 4: Implementar SMOTE para Balanceo
Aplicar SMOTE (Synthetic Minority Oversampling Technique) para generar datos sintéticos de clases minoritarias.


In [7]:
# IMPLEMENTAR SMOTE PARA BALANCEO DE CLASES
class SMOTEAugmentator:
    """Augmentation usando SMOTE para landmarks"""

    def __init__(self):
        self.smote = SMOTE(
            sampling_strategy='auto',  # Balancear automáticamente
            random_state=42,
            k_neighbors=3  # Reducido para datasets pequeños
        )
        self.label_encoder = LabelEncoder()

    def apply_smote(self, df):
        """Aplicar SMOTE al dataset de landmarks"""
        print("🔄 APLICANDO SMOTE PARA BALANCEO")
        print("=" * 40)

        # Preparar datos para SMOTE
        landmark_cols = [col for col in df.columns
                        if col not in ['activity', 'video_file', 'frame_number']]

        X = df[landmark_cols].values
        y = self.label_encoder.fit_transform(df['activity'].values)

        print(f"📊 ANTES DE SMOTE:")
        unique, counts = np.unique(y, return_counts=True)
        activities = self.label_encoder.inverse_transform(unique)
        for activity, count in zip(activities, counts):
            print(f"   {activity.replace('_', ' ').title()}: {count:,} samples")

        # Aplicar SMOTE
        try:
            X_resampled, y_resampled = self.smote.fit_resample(X, y)

            print(f"\n🔄 SMOTE APLICADO EXITOSAMENTE")
            print(f"📊 DESPUÉS DE SMOTE:")

            unique_res, counts_res = np.unique(y_resampled, return_counts=True)
            activities_res = self.label_encoder.inverse_transform(unique_res)

            total_original = len(X)
            total_augmented = len(X_resampled)

            smote_generated = {}
            for activity, count_new in zip(activities_res, counts_res):
                # Contar originales
                original_count = len(df[df['activity'] == activity])
                generated = count_new - original_count
                smote_generated[activity] = max(0, generated)

                print(f"   {activity.replace('_', ' ').title()}: {original_count:,} → {count_new:,} (+{generated:,})")

            print(f"\n📈 RESUMEN SMOTE:")
            print(f"   Frames originales: {total_original:,}")
            print(f"   Frames después SMOTE: {total_augmented:,}")
            print(f"   Frames generados: {total_augmented - total_original:,}")

            # Crear DataFrame con datos aumentados
            df_smote = self._create_smote_dataframe(X_resampled, y_resampled, landmark_cols)

            return df_smote, smote_generated

        except Exception as e:
            print(f"❌ Error aplicando SMOTE: {e}")
            return None, None

    def _create_smote_dataframe(self, X_resampled, y_resampled, landmark_cols):
        """Crear DataFrame con datos SMOTE"""
        # Crear DataFrame base
        df_smote = pd.DataFrame(X_resampled, columns=landmark_cols)

        # Añadir actividades decodificadas
        activities_decoded = self.label_encoder.inverse_transform(y_resampled)
        df_smote['activity'] = activities_decoded

        # Añadir metadata sintética
        df_smote['video_file'] = [f"SMOTE_{activity}_{i:04d}.synthetic"
                                 for i, activity in enumerate(activities_decoded)]
        df_smote['frame_number'] = range(len(df_smote))
        df_smote['augmentation_type'] = 'SMOTE'

        return df_smote

# Crear augmentador SMOTE
smote_augmentator = SMOTEAugmentator()

# Aplicar SMOTE
if base_df is not None:
    df_smote, smote_stats = smote_augmentator.apply_smote(base_df)

    if df_smote is not None:
        print(f"\n✅ SMOTE COMPLETADO")
        print(f"🎯 Dataset balanceado generado: {len(df_smote):,} frames")
else:
    print("❌ No hay datos base para SMOTE")


🔄 APLICANDO SMOTE PARA BALANCEO
📊 ANTES DE SMOTE:
   Caminar Hacia: 991 samples
   Caminar Regreso: 1,041 samples
   Girar: 1,089 samples
   Ponerse Pie: 809 samples
   Sentarse: 645 samples

🔄 SMOTE APLICADO EXITOSAMENTE
📊 DESPUÉS DE SMOTE:
   Caminar Hacia: 991 → 1,089 (+98)
   Caminar Regreso: 1,041 → 1,089 (+48)
   Girar: 1,089 → 1,089 (+0)
   Ponerse Pie: 809 → 1,089 (+280)
   Sentarse: 645 → 1,089 (+444)

📈 RESUMEN SMOTE:
   Frames originales: 4,575
   Frames después SMOTE: 5,445
   Frames generados: 870

✅ SMOTE COMPLETADO
🎯 Dataset balanceado generado: 5,445 frames


## Paso 5: Augmentation Espacial (Rotaciones)
Implementar rotaciones espaciales de landmarks para simular diferentes ángulos de cámara.


In [9]:
# AUGMENTATION ESPACIAL - ROTACIONES PARA ROBUSTEZ
class SpatialAugmentator:
    """Augmentation espacial para mejorar robustez del modelo"""

    def __init__(self):
        self.rotation_angles = [10, -10, 20, -20]  # Rotaciones suaves

    def rotate_for_robustness(self, df, samples_per_activity=100):
        """Aplicar rotaciones para mejorar robustez, no para balanceo"""
        print("🔀 AUGMENTATION ESPACIAL - ROBUSTEZ DEL MODELO")
        print("=" * 50)

        print("💡 OBJETIVO: Mejorar robustez ante variaciones de ángulo de cámara")
        print("🎯 ESTRATEGIA: Generar variaciones espaciales de muestras existentes")

        augmented_dataframes = []
        total_generated = 0

        for activity in df['activity'].unique():
            activity_data = df[df['activity'] == activity].copy()
            activity_generated = 0

            print(f"\n   🎬 {activity.replace('_', ' ').title()}:")

            for angle in self.rotation_angles:
                # Seleccionar muestra pequeña para rotar
                sample_size = min(samples_per_activity // len(self.rotation_angles), len(activity_data) // 10)

                if sample_size > 0:
                    sample_data = activity_data.sample(n=sample_size, random_state=42+angle)

                    # Aplicar rotación
                    rotated_data = self._apply_rotation(sample_data, angle, activity)

                    if rotated_data is not None:
                        augmented_dataframes.append(rotated_data)
                        activity_generated += len(rotated_data)
                        total_generated += len(rotated_data)

                        print(f"      ✅ Rotación {angle:+3d}°: {len(rotated_data):,} frames")

            print(f"   📊 Total generado: {activity_generated:,} frames")

        if augmented_dataframes:
            df_rotated = pd.concat(augmented_dataframes, ignore_index=True)

            print(f"\n📊 ROTACIONES PARA ROBUSTEZ COMPLETADAS:")
            print(f"   Total frames adicionales: {total_generated:,}")
            print(f"   Propósito: Mejorar invarianza espacial")

            return df_rotated
        else:
            print("⚠️ No se generaron rotaciones")
            return None

    def _apply_rotation(self, df, angle_degrees, activity):
        """Aplicar rotación espacial a landmarks"""
        try:
            angle_rad = np.radians(angle_degrees)
            cos_a = np.cos(angle_rad)
            sin_a = np.sin(angle_rad)

            rotated_df = df.copy()

            # Rotar solo landmarks de torso superior (más efectivo)
            landmarks_to_rotate = ['L_shoulder', 'R_shoulder', 'L_elbow', 'R_elbow', 'L_wrist', 'R_wrist']

            for landmark in landmarks_to_rotate:
                x_col = f"{landmark}_x"
                y_col = f"{landmark}_y"

                if x_col in rotated_df.columns and y_col in rotated_df.columns:
                    # Centrar coordenadas
                    x_centered = rotated_df[x_col] - 0.5
                    y_centered = rotated_df[y_col] - 0.5

                    # Aplicar rotación
                    x_rot = x_centered * cos_a - y_centered * sin_a
                    y_rot = x_centered * sin_a + y_centered * cos_a

                    # Recentrar
                    rotated_df[x_col] = np.clip(x_rot + 0.5, 0, 1)
                    rotated_df[y_col] = np.clip(y_rot + 0.5, 0, 1)

            # Actualizar metadata
            rotated_df['video_file'] = [f"ROT{angle_degrees:+03d}_{original}" for original in rotated_df['video_file']]
            rotated_df['augmentation_type'] = f'spatial_rotation_{angle_degrees}deg'

            return rotated_df

        except Exception as e:
            print(f"❌ Error rotación {angle_degrees}°: {e}")
            return None

# EJECUTAR ROTACIONES CORREGIDAS
spatial_augmentator = SpatialAugmentator()

if 'df_smote' in locals() and df_smote is not None:
    # Usar datos SMOTE como base para rotaciones
    df_rotated = spatial_augmentator.rotate_for_robustness(df_smote, samples_per_activity=100)
    print(f"\n✅ ROTACIONES DE ROBUSTEZ APLICADAS AL DATASET BALANCEADO")
else:
    print("❌ Primero debe completarse SMOTE")


🔀 AUGMENTATION ESPACIAL - ROBUSTEZ DEL MODELO
💡 OBJETIVO: Mejorar robustez ante variaciones de ángulo de cámara
🎯 ESTRATEGIA: Generar variaciones espaciales de muestras existentes

   🎬 Caminar Hacia:
      ✅ Rotación +10°: 25 frames
      ✅ Rotación -10°: 25 frames
      ✅ Rotación +20°: 25 frames
      ✅ Rotación -20°: 25 frames
   📊 Total generado: 100 frames

   🎬 Girar:
      ✅ Rotación +10°: 25 frames
      ✅ Rotación -10°: 25 frames
      ✅ Rotación +20°: 25 frames
      ✅ Rotación -20°: 25 frames
   📊 Total generado: 100 frames

   🎬 Ponerse Pie:
      ✅ Rotación +10°: 25 frames
      ✅ Rotación -10°: 25 frames
      ✅ Rotación +20°: 25 frames
      ✅ Rotación -20°: 25 frames
   📊 Total generado: 100 frames

   🎬 Sentarse:
      ✅ Rotación +10°: 25 frames
      ✅ Rotación -10°: 25 frames
      ✅ Rotación +20°: 25 frames
      ✅ Rotación -20°: 25 frames
   📊 Total generado: 100 frames

   🎬 Caminar Regreso:
      ✅ Rotación +10°: 25 frames
      ✅ Rotación -10°: 25 frames
      

## Paso 6: Augmentation Temporal (Interpolación)
Generar frames intermedios usando interpolación cúbica para crear secuencias más densas.


In [13]:
# AUGMENTATION TEMPORAL - INTERPOLACIÓN CÚBICA (CORREGIDA)
class TemporalAugmentator:
    """Augmentation temporal para secuencias de landmarks"""

    def __init__(self):
        pass

    def interpolate_sequences(self, df, target_frames=300):
        """Generar frames intermedios por interpolación"""
        print("⏱️ AUGMENTATION TEMPORAL - INTERPOLACIÓN")
        print("=" * 50)

        print("💡 OBJETIVO: Crear transiciones más suaves entre frames")
        print("🎯 MÉTODO: Interpolación cúbica en secuencias existentes")

        # Verificar si el dataset tiene actividades
        if 'activity' not in df.columns:
            print("❌ No se encontró columna 'activity'")
            return None

        unique_activities = df['activity'].unique()

        if len(unique_activities) == 0:
            print("❌ No se encontraron actividades")
            return None

        augmented_sequences = []
        total_generated = 0

        frames_per_activity = target_frames // len(unique_activities)

        print(f"📊 PARÁMETROS:")
        print(f"   Actividades encontradas: {len(unique_activities)}")
        print(f"   Target por actividad: {frames_per_activity} frames")

        for activity in unique_activities:
            activity_data = df[df['activity'] == activity].copy()
            activity_generated = 0

            print(f"\n   🎬 {activity.replace('_', ' ').title()}: Target +{frames_per_activity:,} frames")

            # Obtener videos únicos para esta actividad
            unique_videos = activity_data['video_file'].unique()
            print(f"      📹 Videos disponibles: {len(unique_videos)}")

            for video_file in unique_videos:
                if activity_generated >= frames_per_activity:
                    break

                video_sequence = activity_data[activity_data['video_file'] == video_file].copy()
                video_sequence = video_sequence.sort_values('frame_number').reset_index(drop=True)

                # Solo interpolar videos cortos (necesitan más densidad)
                if len(video_sequence) < 100:  # Videos cortos
                    interpolated = self._interpolate_video(video_sequence, activity)

                    if interpolated is not None:
                        new_frames = len(interpolated) - len(video_sequence)
                        if new_frames > 0:
                            augmented_sequences.append(interpolated)
                            activity_generated += new_frames
                            total_generated += new_frames

                            print(f"      ✅ {video_file}: {len(video_sequence):,} → {len(interpolated):,} (+{new_frames:,})")
                else:
                    print(f"      ➖ {video_file}: {len(video_sequence):,} frames (suficientemente denso)")

            print(f"   📊 Total generado para {activity}: {activity_generated:,} frames")

        if augmented_sequences:
            df_interpolated = pd.concat(augmented_sequences, ignore_index=True)

            print(f"\n📊 INTERPOLACIÓN COMPLETADA:")
            print(f"   Total frames generados: {total_generated:,}")
            print(f"   Secuencias procesadas: {len(augmented_sequences):,}")
            print(f"   Propósito: Transiciones más suaves")

            return df_interpolated
        else:
            print(f"\n💡 NO SE REQUIERE INTERPOLACIÓN:")
            print(f"   Todos los videos tienen densidad suficiente (≥100 frames)")
            print(f"   Dataset actual es apropiado para entrenamiento")
            return None

    def _interpolate_video(self, video_df, activity):
        """Interpolar una secuencia de video individual"""
        try:
            landmark_cols = [col for col in video_df.columns
                           if col not in ['activity', 'video_file', 'frame_number', 'augmentation_type']]

            original_length = len(video_df)
            # Aumentar 50% la densidad para videos cortos
            new_length = int(original_length * 1.5)

            # Índices originales y nuevos
            original_idx = np.arange(original_length)
            new_idx = np.linspace(0, original_length-1, new_length)

            # Datos interpolados
            interpolated_data = {}

            for col in landmark_cols:
                values = video_df[col].values

                # Interpolación cúbica suave
                cs = CubicSpline(original_idx, values, extrapolate=False)
                interpolated_values = cs(new_idx)

                # Validar rangos según tipo de coordenada
                if col.endswith('_x') or col.endswith('_y'):
                    interpolated_values = np.clip(interpolated_values, 0, 1)
                elif col.endswith('_visibility'):
                    interpolated_values = np.clip(interpolated_values, 0, 1)
                elif col.endswith('_z'):
                    # Z más permisivo pero limitado
                    interpolated_values = np.clip(interpolated_values, -1.5, 1.5)

                interpolated_data[col] = interpolated_values

            # Crear DataFrame interpolado
            df_interp = pd.DataFrame(interpolated_data)
            df_interp['activity'] = activity
            df_interp['video_file'] = f"INTERP_{video_df['video_file'].iloc[0]}"
            df_interp['frame_number'] = range(len(df_interp))
            df_interp['augmentation_type'] = 'temporal_interpolation'

            return df_interp

        except Exception as e:
            print(f"❌ Error interpolando video: {e}")
            return None

# Crear augmentador temporal CORREGIDO
temporal_augmentator = TemporalAugmentator()

# APLICAR INTERPOLACIÓN AL DATASET BASE (no SMOTE)
if 'base_df' in globals() and base_df is not None:
    print("🎯 USANDO DATASET BASE PARA INTERPOLACIÓN")
    df_interpolated = temporal_augmentator.interpolate_sequences(base_df, target_frames=300)

    if df_interpolated is not None:
        print(f"\n✅ INTERPOLACIÓN TEMPORAL COMPLETADA")
    else:
        print(f"\n💡 INTERPOLACIÓN OMITIDA - Dataset tiene densidad apropiada")
else:
    print("❌ No hay dataset base para interpolación")
    df_interpolated = None


🎯 USANDO DATASET BASE PARA INTERPOLACIÓN
⏱️ AUGMENTATION TEMPORAL - INTERPOLACIÓN
💡 OBJETIVO: Crear transiciones más suaves entre frames
🎯 MÉTODO: Interpolación cúbica en secuencias existentes
📊 PARÁMETROS:
   Actividades encontradas: 5
   Target por actividad: 60 frames

   🎬 Caminar Hacia: Target +60 frames
      📹 Videos disponibles: 9
      ✅ caminar_hacia_005.mp4: 95 → 142 (+47)
      ➖ caminar_hacia_009.mp4: 115 frames (suficientemente denso)
      ✅ caminar_hacia_003.mp4: 95 → 142 (+47)
   📊 Total generado para caminar_hacia: 94 frames

   🎬 Girar: Target +60 frames
      📹 Videos disponibles: 8
      ➖ girar_006.mp4: 124 frames (suficientemente denso)
      ➖ girar_003.mp4: 122 frames (suficientemente denso)
      ➖ girar_007.mp4: 139 frames (suficientemente denso)
      ➖ girar_001.mp4: 134 frames (suficientemente denso)
      ➖ girar_002.mp4: 130 frames (suficientemente denso)
      ➖ girar_005.mp4: 152 frames (suficientemente denso)
      ➖ girar_004.mp4: 156 frames (suficie

## Paso 7: Combinar Dataset Final Aumentado
Combinar todos los datos aumentados (original + SMOTE + rotaciones + interpolación) en dataset final.


In [16]:
# COMBINAR TODOS LOS DATOS AUMENTADOS - VERSIÓN FINAL
def create_final_augmented_dataset():
    """Combinar todos los datasets aumentados en versión final"""
    print("🔗 CREANDO DATASET FINAL AUMENTADO")
    print("=" * 50)

    all_dataframes = []
    technique_summary = {}

    # 1. Datos originales limpios
    if 'base_df' in globals() and base_df is not None:
        original_data = base_df.copy()
        original_data['augmentation_type'] = 'original'
        original_data['source'] = 'real_video'
        all_dataframes.append(original_data)
        technique_summary['original'] = len(original_data)
        print(f"   ✅ Datos originales: {len(original_data):,} frames")

    # 2. Datos SMOTE (sintéticos de balanceo)
    if 'df_smote' in globals() and df_smote is not None:
        # Filtrar solo los sintéticos (identificar por nombres de archivo)
        smote_data = df_smote[df_smote['video_file'].str.contains('SMOTE_', na=False)].copy()
        if not smote_data.empty:
            smote_data['source'] = 'smote_synthetic'
            all_dataframes.append(smote_data)
            technique_summary['SMOTE'] = len(smote_data)
            print(f"   ✅ SMOTE sintético: {len(smote_data):,} frames")

    # 3. Datos rotados (robustez espacial)
    if 'df_rotated' in globals() and df_rotated is not None:
        rotated_data = df_rotated.copy()
        rotated_data['source'] = 'spatial_augmentation'
        all_dataframes.append(rotated_data)
        technique_summary['spatial_rotation'] = len(rotated_data)
        print(f"   ✅ Rotaciones espaciales: {len(rotated_data):,} frames")

    # 4. Datos interpolados (densidad temporal)
    if 'df_interpolated' in globals() and df_interpolated is not None:
        interpolated_data = df_interpolated.copy()
        interpolated_data['source'] = 'temporal_augmentation'
        all_dataframes.append(interpolated_data)
        technique_summary['temporal_interpolation'] = len(interpolated_data)
        print(f"   ✅ Interpolación temporal: {len(interpolated_data):,} frames")

    if not all_dataframes:
        print("❌ No hay datasets para combinar")
        return None

    # Combinar todos los DataFrames
    df_master = pd.concat(all_dataframes, ignore_index=True)

    print(f"\n📊 DATASET MAESTRO CREADO:")
    print(f"   📈 Total frames: {len(df_master):,}")
    print(f"   📹 Videos únicos: {df_master['video_file'].nunique():,}")
    print(f"   🔧 Técnicas aplicadas: {len(technique_summary)}")

    # Análisis de distribución final
    print(f"\n🎯 DISTRIBUCIÓN FINAL POR ACTIVIDAD:")
    final_distribution = df_master['activity'].value_counts()

    for activity, count in final_distribution.items():
        percentage = count / len(df_master) * 100
        print(f"   {activity.replace('_', ' ').title()}: {count:,} frames ({percentage:.1f}%)")

    # Calcular balance final
    balance_ratio = final_distribution.min() / final_distribution.max()
    balance_improvement = ((balance_ratio - 0.59) / 0.59) * 100

    print(f"\n⚖️ ANÁLISIS DE BALANCE:")
    print(f"   Balance original (EDA): 0.59")
    print(f"   Balance final: {balance_ratio:.3f}")
    print(f"   Mejora: {balance_improvement:+.1f}%")
    print(f"   Evaluación: {'✅ EXCELENTE' if balance_ratio > 0.95 else '✅ BUENO' if balance_ratio > 0.85 else '📈 MEJORADO'}")

    # Distribución por fuente de datos
    print(f"\n📊 DISTRIBUCIÓN POR FUENTE:")
    source_distribution = df_master['source'].value_counts()
    for source, count in source_distribution.items():
        percentage = count / len(df_master) * 100
        print(f"   {source.replace('_', ' ').title()}: {count:,} frames ({percentage:.1f}%)")

    return df_master, technique_summary, balance_ratio

# Ejecutar combinación final
final_result = create_final_augmented_dataset()

if final_result[0] is not None:
    df_complete_augmented, tech_summary, final_balance = final_result
    print(f"\n🎉 DATASET FINAL AUMENTADO CREADO EXITOSAMENTE")
else:
    print(f"\n❌ Error creando dataset final")


🔗 CREANDO DATASET FINAL AUMENTADO
   ✅ Datos originales: 4,575 frames
   ✅ SMOTE sintético: 5,445 frames
   ✅ Rotaciones espaciales: 500 frames
   ✅ Interpolación temporal: 886 frames

📊 DATASET MAESTRO CREADO:
   📈 Total frames: 11,406
   📹 Videos únicos: 5,997
   🔧 Técnicas aplicadas: 4

🎯 DISTRIBUCIÓN FINAL POR ACTIVIDAD:
   Caminar Hacia: 2,464 frames (21.6%)
   Caminar Regreso: 2,368 frames (20.8%)
   Girar: 2,278 frames (20.0%)
   Ponerse Pie: 2,227 frames (19.5%)
   Sentarse: 2,069 frames (18.1%)

⚖️ ANÁLISIS DE BALANCE:
   Balance original (EDA): 0.59
   Balance final: 0.840
   Mejora: +42.3%
   Evaluación: 📈 MEJORADO

📊 DISTRIBUCIÓN POR FUENTE:
   Smote Synthetic: 5,445 frames (47.7%)
   Real Video: 4,575 frames (40.1%)
   Temporal Augmentation: 886 frames (7.8%)
   Spatial Augmentation: 500 frames (4.4%)

🎉 DATASET FINAL AUMENTADO CREADO EXITOSAMENTE


In [18]:
# GUARDAR DATASET MAESTRO FINAL
print("💾 GUARDANDO DATASET MAESTRO AUMENTADO")
print("=" * 45)

# Crear carpeta
os.makedirs('data/augmented', exist_ok=True)

# Guardar archivo principal
dataset_path = 'data/augmented/landmarks_final_augmented.csv'
df_complete_augmented.to_csv(dataset_path, index=False)

# Mostrar información
file_size = os.path.getsize(dataset_path) / (1024*1024)

print(f"✅ DATASET GUARDADO:")
print(f"   📁 {dataset_path}")
print(f"   📊 {len(df_complete_augmented):,} frames")
print(f"   💽 {file_size:.1f} MB")
print(f"   ⚖️ Balance: 0.840")

print(f"\n🎉 NOTEBOOK 3 - DATA AUGMENTATION COMPLETADO")

💾 GUARDANDO DATASET MAESTRO AUMENTADO
✅ DATASET GUARDADO:
   📁 data/augmented/landmarks_final_augmented.csv
   📊 11,406 frames
   💽 13.9 MB
   ⚖️ Balance: 0.840

🎉 NOTEBOOK 3 - DATA AUGMENTATION COMPLETADO
